# Fundamentos de ETL

1) EXTRAÇÃO

In [3]:
import pandas as pd

In [ ]:
# upload de arquivo via código
from google.colab import files
arq = files.upload()

In [5]:
# lendo file, diferenciando os dados que são do tipo date, garantir ordem y-m-d nas datas.
df = pd.read_csv('/content/ocorrencia_2010_2020 (1).csv', parse_dates = ['ocorrencia_dia'], dayfirst = True)
df

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora
0,40211,40211,INCIDENTE,RIO DE JANEIRO,RJ,****,2010-03-01,12:00:00
1,40349,40349,INCIDENTE,BELÉM,PA,SBBE,2010-03-01,11:05:00
2,40351,40351,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-03-01,3:00:00
3,39527,39527,ACIDENTE,LUCAS DO RIO VERDE,MT,****,2010-04-01,17:30:00
4,40324,40324,INCIDENTE,PELOTAS,RS,SBPK,2010-05-01,19:25:00
...,...,...,...,...,...,...,...,...
5747,79804,79804,INCIDENTE,CAMPINAS,SP,SBKP,2020-12-29,19:00:00
5748,79757,79757,INCIDENTE GRAVE,LAGOA DA CONFUSÃO,TO,****,2020-12-30,18:30:00
5749,79802,79802,INCIDENTE,RIO DE JANEIRO,RJ,SBGL,2020-12-30,0:54:00
5750,79756,79756,INCIDENTE GRAVE,VICENTINA,MS,****,2020-12-31,9:00:00


In [6]:
# retorno dos tipos de dado existentes em cada coluna
df.dtypes

codigo_ocorrencia                    int64
codigo_ocorrencia2                   int64
ocorrencia_classificacao            object
ocorrencia_cidade                   object
ocorrencia_uf                       object
ocorrencia_aerodromo                object
ocorrencia_dia              datetime64[ns]
ocorrencia_hora                     object
dtype: object

In [8]:
# retorno apenas dos meses
df.ocorrencia_dia.dt.month.reset_index()

,index,ocorrencia_dia
0,0,3
1,1,3
2,2,3
3,3,4
4,4,5
...,...,...
5747,5747,12
5748,5748,12
5749,5749,12
5750,5750,12


In [9]:
# retornando últimas dez linhas
df.tail(10)

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora
5742,79800,79800,INCIDENTE,SÃO PAULO,SP,SBMT,2020-12-28,10:15:00
5743,79824,79824,ACIDENTE,RIO PARANAÍBA,MG,SNRP,2020-12-28,17:00:00
5744,79753,79753,INCIDENTE GRAVE,GOIATUBA,GO,****,2020-12-29,12:00:00
5745,79755,79755,ACIDENTE,MATO RICO,PR,****,2020-12-29,10:30:00
5746,79769,79769,INCIDENTE GRAVE,MANOEL URBANO,AC,SIMB,2020-12-29,18:30:00
5747,79804,79804,INCIDENTE,CAMPINAS,SP,SBKP,2020-12-29,19:00:00
5748,79757,79757,INCIDENTE GRAVE,LAGOA DA CONFUSÃO,TO,****,2020-12-30,18:30:00
5749,79802,79802,INCIDENTE,RIO DE JANEIRO,RJ,SBGL,2020-12-30,0:54:00
5750,79756,79756,INCIDENTE GRAVE,VICENTINA,MS,****,2020-12-31,9:00:00
5751,79844,79844,INCIDENTE,RIO DE JANEIRO,RJ,SBJR,2020-12-31,13:24:00


2) VALIDAÇÃO DA EXTRAÇÃO


In [ ]:
# lib útil pra validação de dados
pip install pandera 

In [15]:
import pandera as pa

In [22]:
df = pd.read_csv('/content/ocorrencia_2010_2020 (1).csv', parse_dates = ['ocorrencia_dia'], dayfirst = True)
df.head(10)

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora
0,40211,40211,INCIDENTE,RIO DE JANEIRO,RJ,****,2010-03-01,12:00:00
1,40349,40349,INCIDENTE,BELÉM,PA,SBBE,2010-03-01,11:05:00
2,40351,40351,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-03-01,3:00:00
3,39527,39527,ACIDENTE,LUCAS DO RIO VERDE,MT,****,2010-04-01,17:30:00
4,40324,40324,INCIDENTE,PELOTAS,RS,SBPK,2010-05-01,19:25:00
5,39807,39807,INCIDENTE,SALVADOR,BA,****,2010-06-01,17:53:00
6,40215,40215,INCIDENTE,COARI,AM,SBUY,2010-07-01,18:40:00
7,39707,39707,INCIDENTE GRAVE,CANUTAMA,AM,****,2010-09-01,12:30:00
8,39156,39156,INCIDENTE GRAVE,CASCAVEL,PR,SBCA,2010-10-01,23:15:00
9,39711,39711,INCIDENTE GRAVE,PARÁ DE MINAS,MG,****,2010-10-01,20:00:00


In [80]:
# esquema a ser validado --> identificação de erros
schema = pa.DataFrameSchema(
    columns = {
        'codigo_ocorrencia': pa.Column(pa.Int),
        'codigo_ocorrencia2': pa.Column(pa.Int),
        'ocorrencia_classificacao': pa.Column(pa.String),
        'ocorrencia_cidade': pa.Column(pa.String),
        'ocorrencia_uf': pa.Column(pa.String),
        'ocorrencia_aerodromo': pa.Column(pa.String),
        'ocorrencia_dia': pa.Column(pa.DateTime),
        'ocorrencia_hora': pa.Column(pa.String, nullable = True)
    }
)

In [30]:
schema.validate(df)

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora
0,40211,40211,INCIDENTE,RIO DE JANEIRO,RJ,****,2010-03-01,12:00:00
1,40349,40349,INCIDENTE,BELÉM,PA,SBBE,2010-03-01,11:05:00
2,40351,40351,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-03-01,3:00:00
3,39527,39527,ACIDENTE,LUCAS DO RIO VERDE,MT,****,2010-04-01,17:30:00
4,40324,40324,INCIDENTE,PELOTAS,RS,SBPK,2010-05-01,19:25:00
...,...,...,...,...,...,...,...,...
5747,79804,79804,INCIDENTE,CAMPINAS,SP,SBKP,2020-12-29,19:00:00
5748,79757,79757,INCIDENTE GRAVE,LAGOA DA CONFUSÃO,TO,****,2020-12-30,18:30:00
5749,79802,79802,INCIDENTE,RIO DE JANEIRO,RJ,SBGL,2020-12-30,0:54:00
5750,79756,79756,INCIDENTE GRAVE,VICENTINA,MS,****,2020-12-31,9:00:00


3) LIMPEZA DE DADOS

3.1 Fazendo algumas alterações

In [37]:
df.loc[0:2]                                  # dados das três primeiras linhas
df.loc[[10,40]]                              # dados das linhas 10 e 40
df.loc[:, 'ocorrencia_uf'].reset_index()     # todos os dados da coluna ocorrencia_uf

,index,ocorrencia_uf
0,0,RJ
1,1,PA
2,2,RJ
3,3,MT
4,4,RS
...,...,...
5747,5747,SP
5748,5748,TO
5749,5749,RJ
5750,5750,MS


In [38]:
# observar se há repetição de texto/valor
df.codigo_ocorrencia.is_unique    

True

In [39]:
# transformar a coluna em índice para a tabela como um todo
df.set_index('codigo_ocorrencia', inplace = True)

In [40]:
# utilizando codigo_ocorrencia para retornar as infos de uma linha
df.loc[40324] 

codigo_ocorrencia2                        40324
ocorrencia_classificacao              INCIDENTE
ocorrencia_cidade                       PELOTAS
ocorrencia_uf                                RS
ocorrencia_aerodromo                       SBPK
ocorrencia_dia              2010-05-01 00:00:00
ocorrencia_hora                        19:25:00
Name: 40324, dtype: object

In [41]:
# voltar ao indice original
df.reset_index(drop = True, inplace = True)
df.head()

,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora
0,40211,INCIDENTE,RIO DE JANEIRO,RJ,****,2010-03-01,12:00:00
1,40349,INCIDENTE,BELÉM,PA,SBBE,2010-03-01,11:05:00
2,40351,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-03-01,3:00:00
3,39527,ACIDENTE,LUCAS DO RIO VERDE,MT,****,2010-04-01,17:30:00
4,40324,INCIDENTE,PELOTAS,RS,SBPK,2010-05-01,19:25:00


In [44]:
#### alterando dados

# deixando vazia a célula com asterisco da primeira linha de ocorrencia_aerodromo
df.loc[0,'ocorrencia_aerodromo'] = ''
df.head(1)

''' procedimentos abruptos:
# alterando todos os dados de uma linha
df.loc[1] = 20  
df.head(2)

# alterando todos os dados de uma coluna
df.loc[:, 'ocorrencia_dia'] = 10  
df.head()
'''

# criando coluna de backup de segurança para não perder dados originais!!!
df['ocorrencia_uf_bkp'] = df.ocorrencia_uf  
df

# alterando a classificação dos incidentes
# as ocorrencias de SP têm todas a classificação "GRAVE"
df.loc[df.ocorrencia_uf == 'SP', ['ocorrencia_classificacao']] = 'GRAVE'
df.loc[df.ocorrencia_uf == 'SP']

,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,ocorrencia_uf_bkp
11,40069,GRAVE,SÃO PAULO,SP,SBMT,2010-10-01,14:50:00,SP
16,39809,GRAVE,SÃO PAULO,SP,****,2010-01-15,15:00:00,SP
18,39828,GRAVE,SANTOS,SP,****,2010-01-15,17:45:00,SP
26,39847,GRAVE,SOROCABA,SP,****,2010-01-20,13:10:00,SP
27,39768,GRAVE,CAMPINAS,SP,****,2010-01-21,20:45:00,SP
...,...,...,...,...,...,...,...,...
5704,79739,GRAVE,SÃO PAULO,SP,SBSP,2020-03-12,14:42:00,SP
5705,79705,GRAVE,SOROCABA,SP,SDCO,2020-04-12,15:30:00,SP
5715,79718,GRAVE,SÃO PAULO,SP,SBMT,2020-11-12,13:50:00,SP
5742,79800,GRAVE,SÃO PAULO,SP,SBMT,2020-12-28,10:15:00,SP


In [46]:
# se esse código for rodado, retornará o dataset original, porque as alterações 
# feitas no bloco anterior dizem respeito apenas ao dataframe

df = pd.read_csv('/content/ocorrencia_2010_2020 (1).csv', parse_dates = ['ocorrencia_dia'], dayfirst = True)
df

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora
0,40211,40211,INCIDENTE,RIO DE JANEIRO,RJ,****,2010-03-01,12:00:00
1,40349,40349,INCIDENTE,BELÉM,PA,SBBE,2010-03-01,11:05:00
2,40351,40351,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-03-01,3:00:00
3,39527,39527,ACIDENTE,LUCAS DO RIO VERDE,MT,****,2010-04-01,17:30:00
4,40324,40324,INCIDENTE,PELOTAS,RS,SBPK,2010-05-01,19:25:00
...,...,...,...,...,...,...,...,...
5747,79804,79804,INCIDENTE,CAMPINAS,SP,SBKP,2020-12-29,19:00:00
5748,79757,79757,INCIDENTE GRAVE,LAGOA DA CONFUSÃO,TO,****,2020-12-30,18:30:00
5749,79802,79802,INCIDENTE,RIO DE JANEIRO,RJ,SBGL,2020-12-30,0:54:00
5750,79756,79756,INCIDENTE GRAVE,VICENTINA,MS,****,2020-12-31,9:00:00


3.2 Limpeza de dados

In [47]:
# agora sim, depois de fazer as modificações desejadas, porém mudando de ideia
# e voltando ao dataset original

### limpando dados

''' Utilizando filtro de excel no csv analisado percebemos que os seguintes dados
precisam ser limpados do dataframe:

ocorrencia_uf
**

ocorrencia_aerodromo
****
*****
####
####!

ocorrencia_hora
NULL
'''
# opção 1 -- alterando de modo específico para <NA>
# df.loc[df.'ocorrencia_aerodromo' == '****', ['ocorrencia_aerodromo']] = pd.NA

# opção 2 -- alterando no geral
df.replace(['**', '****', '*****', '####', '####!', 'NULL'], pd.NA, inplace = True)

In [48]:
# saber quantos NA's há no dataframe
df.isna().sum()       # ou df.isnull().sum()  

codigo_ocorrencia              0
codigo_ocorrencia2             0
ocorrencia_classificacao       0
ocorrencia_cidade              0
ocorrencia_uf                  0
ocorrencia_aerodromo        2344
ocorrencia_dia                 0
ocorrencia_hora                1
dtype: int64

In [49]:
# replace os NA's com alguma coisa
# atenção à necessidade de usar o "inplace = True"
df.fillna(0, inplace = True)

In [50]:
# replace NA's ou alguma coisa somente em algumas colunas
df.fillna(value = {'ocorrencia_classificacao':30}, inplace = True)

In [51]:
# exclusão de uma coluna de backup que não terá mais uso
# obs. utiliza-se "axis = 1" para especificar que se trata da coluna. Isso porque,
# por padrão, se nenhum parâmetro for escrito o python considera "axis = 0" que 
# se refere ao eixo horizontal, i.e., às linhas.
df['ocorrencia_hora_bkp'] = df.ocorrencia_hora
df.drop(['ocorrencia_hora_bkp'],  axis = 1, inplace = True)

In [52]:
# remove a LINHA TODA em que um valor nulo se encontrava
# df.dropna()

# remove linhas duplicadas
df.drop_duplicates(inplace = True)

4) PARTE FINAL: TRANSFORMAÇÃO


In [53]:
import pandas as pd
import pandera as pa

In [57]:
# adicionando na leitura do dataframe (e antes da validação) o trabalho de 
# limpeza já feito. (Apenas porque nesse caso é possível fazer isso.)

valores_ausentes = ['**', '****', '*****', '####', '####!', 'NULL']
df = pd.read_csv('/content/ocorrencia_2010_2020 (1).csv', parse_dates = ['ocorrencia_dia'], dayfirst = True, na_values = valores_ausentes)

schema = pa.DataFrameSchema(
    columns = {
        'codigo_ocorrencia': pa.Column(pa.Int),
        'codigo_ocorrencia2': pa.Column(pa.Int),
        'ocorrencia_classificacao': pa.Column(pa.String),
        'ocorrencia_cidade': pa.Column(pa.String),
        'ocorrencia_uf': pa.Column(pa.String, nullable = True),
        'ocorrencia_aerodromo': pa.Column(pa.String, nullable = True),
        'ocorrencia_dia': pa.Column(pa.DateTime),
        'ocorrencia_hora': pa.Column(pa.String, nullable = True)
    }
)

In [58]:
schema.validate(df)

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora
0,40211,40211,INCIDENTE,RIO DE JANEIRO,RJ,NaN,2010-03-01,12:00:00
1,40349,40349,INCIDENTE,BELÉM,PA,SBBE,2010-03-01,11:05:00
2,40351,40351,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-03-01,3:00:00
3,39527,39527,ACIDENTE,LUCAS DO RIO VERDE,MT,NaN,2010-04-01,17:30:00
4,40324,40324,INCIDENTE,PELOTAS,RS,SBPK,2010-05-01,19:25:00
...,...,...,...,...,...,...,...,...
5747,79804,79804,INCIDENTE,CAMPINAS,SP,SBKP,2020-12-29,19:00:00
5748,79757,79757,INCIDENTE GRAVE,LAGOA DA CONFUSÃO,TO,NaN,2020-12-30,18:30:00
5749,79802,79802,INCIDENTE,RIO DE JANEIRO,RJ,SBGL,2020-12-30,0:54:00
5750,79756,79756,INCIDENTE GRAVE,VICENTINA,MS,NaN,2020-12-31,9:00:00


In [ ]:
# utilização da variavel filtro para facilitar leitura e localização de infos
# ex. localizando células com valor nulo
filtro = df.ocorrencia_aerodromo.is_null()
df.loc[filtro]

# count por default não conta células nulas
df.count()

In [62]:
### aprimorando filtros

# ocorrências no rio de janeiro + nome das cidades
filtro = df.ocorrencia_uf == 'RJ'
df.loc[filtro, ['ocorrencia_uf', 'ocorrencia_cidade']]

,ocorrencia_uf,ocorrencia_cidade
0,RJ,RIO DE JANEIRO
2,RJ,RIO DE JANEIRO
19,RJ,RIO DE JANEIRO
23,RJ,RIO DE JANEIRO
24,RJ,RIO DE JANEIRO
...,...,...
5706,RJ,RIO DE JANEIRO
5710,RJ,RIO DE JANEIRO
5716,RJ,ANGRA DOS REIS
5749,RJ,RIO DE JANEIRO


In [63]:
# todas as infos sobre ocorrências graves
filtro = df.ocorrencia_classificacao == 'INCIDENTE GRAVE'
df.loc[filtro]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora
7,39707,39707,INCIDENTE GRAVE,CANUTAMA,AM,NaN,2010-09-01,12:30:00
8,39156,39156,INCIDENTE GRAVE,CASCAVEL,PR,SBCA,2010-10-01,23:15:00
9,39711,39711,INCIDENTE GRAVE,PARÁ DE MINAS,MG,NaN,2010-10-01,20:00:00
29,39709,39709,INCIDENTE GRAVE,CURITIBA,PR,SBBI,2010-01-23,16:36:00
35,39487,39487,INCIDENTE GRAVE,PALMAS,TO,SBPJ,2010-01-26,16:50:00
...,...,...,...,...,...,...,...,...
5741,79754,79754,INCIDENTE GRAVE,NOVA MARINGÁ,MT,NaN,2020-12-28,13:00:00
5744,79753,79753,INCIDENTE GRAVE,GOIATUBA,GO,NaN,2020-12-29,12:00:00
5746,79769,79769,INCIDENTE GRAVE,MANOEL URBANO,AC,SIMB,2020-12-29,18:30:00
5748,79757,79757,INCIDENTE GRAVE,LAGOA DA CONFUSÃO,TO,NaN,2020-12-30,18:30:00


In [64]:
# ocorrências graves no estado de Minas Gerais
filtro1 = df.ocorrencia_classificacao == 'INCIDENTE GRAVE'
filtro2 = df.ocorrencia_uf == 'MG'
df.loc[filtro1 & filtro2]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora
9,39711,39711,INCIDENTE GRAVE,PARÁ DE MINAS,MG,NaN,2010-10-01,20:00:00
75,40316,40316,INCIDENTE GRAVE,TIMÓTEO,MG,NaN,2010-02-26,14:40:00
168,40621,40621,INCIDENTE GRAVE,POÇOS DE CALDAS,MG,SBPC,2010-05-05,21:05:00
255,41670,41670,INCIDENTE GRAVE,IPATINGA,MG,NaN,2010-06-28,12:45:00
537,43474,43474,INCIDENTE GRAVE,BELO HORIZONTE,MG,SBPR,2010-12-20,22:30:00
...,...,...,...,...,...,...,...,...
5448,79341,79341,INCIDENTE GRAVE,VARGINHA,MG,NaN,2020-05-18,20:00:00
5694,79691,79691,INCIDENTE GRAVE,BELO HORIZONTE,MG,SBBH,2020-11-26,20:10:00
5718,79722,79722,INCIDENTE GRAVE,PERDIZES,MG,NaN,2020-12-12,17:22:00
5720,79726,79726,INCIDENTE GRAVE,BELO HORIZONTE,MG,SBPR,2020-12-13,13:41:00


In [65]:
# ocorrências graves OU no estado do Amazonas
filtro1 = df.ocorrencia_classificacao == 'INCIDENTE GRAVE'
filtro2 = df.ocorrencia_uf == 'AM'
df.loc[filtro1 | filtro2]


,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora
6,40215,40215,INCIDENTE,COARI,AM,SBUY,2010-07-01,18:40:00
7,39707,39707,INCIDENTE GRAVE,CANUTAMA,AM,NaN,2010-09-01,12:30:00
8,39156,39156,INCIDENTE GRAVE,CASCAVEL,PR,SBCA,2010-10-01,23:15:00
9,39711,39711,INCIDENTE GRAVE,PARÁ DE MINAS,MG,NaN,2010-10-01,20:00:00
15,39315,39315,ACIDENTE,CANUTAMA,AM,NaN,2010-01-15,21:22:00
...,...,...,...,...,...,...,...,...
5741,79754,79754,INCIDENTE GRAVE,NOVA MARINGÁ,MT,NaN,2020-12-28,13:00:00
5744,79753,79753,INCIDENTE GRAVE,GOIATUBA,GO,NaN,2020-12-29,12:00:00
5746,79769,79769,INCIDENTE GRAVE,MANOEL URBANO,AC,SIMB,2020-12-29,18:30:00
5748,79757,79757,INCIDENTE GRAVE,LAGOA DA CONFUSÃO,TO,NaN,2020-12-30,18:30:00


In [67]:
'''forma mais simples de fazer o código com .isin:

# ((ocorrências graves) ou (ocorrências comuns)) no estado de Goiás
filtro1 = (df.ocorrencia_classificacao.isin == (['INCIDENTE GRAVE', 'INCIDENTE'])
filtro2 = df.ocorrencia_uf == 'GO'
df.loc[filtro1 & filtro2]
'''
# ((ocorrências graves) ou (ocorrências comuns)) no estado de Goiás
filtro1 = (df.ocorrencia_classificacao == 'INCIDENTE GRAVE') | (df.ocorrencia_classificacao == 'INCIDENTE')
filtro2 = df.ocorrencia_uf == 'GO'
df.loc[filtro1 & filtro2]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora
43,39787,39787,INCIDENTE,GOIÂNIA,GO,SBGO,2010-01-31,21:00:00
60,39365,39365,INCIDENTE,GOIÂNIA,GO,NaN,2010-12-02,16:48:00
121,40269,40269,INCIDENTE GRAVE,ANÁPOLIS,GO,SWNS,2010-07-04,19:30:00
125,40213,40213,INCIDENTE GRAVE,GOIÂNIA,GO,SWNV,2010-09-04,21:00:00
157,40819,40819,INCIDENTE,NIQUELÂNDIA,GO,NaN,2010-04-28,18:30:00
...,...,...,...,...,...,...,...,...
5458,79351,79351,INCIDENTE GRAVE,ITUMBIARA,GO,SBIT,2020-05-26,10:30:00
5548,79491,79491,INCIDENTE GRAVE,ANÁPOLIS,GO,SWNS,2020-08-13,13:00:00
5701,79750,79750,INCIDENTE,GOIÂNIA,GO,SWNV,2020-01-12,11:30:00
5722,79738,79738,INCIDENTE,ITUMBIARA,GO,NaN,2020-12-16,20:30:00


In [71]:
# análise parcial
# ex. todas as ocorrências cuja cidade começa com C
filtro = df.ocorrencia_cidade.str[0] == 'C'
df.loc[filtro, 'codigo_ocorrencia'].reset_index()

,index,codigo_ocorrencia
0,6,40215
1,7,39707
2,8,39156
3,15,39315
4,20,40310
...,...,...
820,5661,79649
821,5663,79652
822,5680,79683
823,5696,79692


In [72]:
# ex. todas as ocorrências cuja cidade termina com 'MA'
filtro = df.ocorrencia_cidade.str[-2:] == 'MA'
df.loc[filtro, 'codigo_ocorrencia']

7       39707
15      39315
408     43062
688     44198
1035    51785
1377    45939
1401    46180
2099    49113
2102    48799
2655    51347
2855    52340
2966    52832
2997    52979
3040    52984
3092    53596
3157    53223
3171    53265
3189    53271
3682    65414
3725    65633
3881    66265
4229    77371
4311    77612
4574    78080
4986    78867
5070    78923
5080    78782
5405    79282
5440    79338
5474    79400
Name: codigo_ocorrencia, dtype: int64

In [73]:
# ex. todas as ocorrências cuja uf CONTÉM 'S' (em qualquer posição)
filtro = df.ocorrencia_uf.str.contains('S')
df.loc[filtro]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora
4,40324,40324,INCIDENTE,PELOTAS,RS,SBPK,2010-05-01,19:25:00
10,39789,39789,INCIDENTE,SÃO PEDRO DO SUL,RS,NaN,2010-10-01,21:30:00
11,40069,40069,ACIDENTE,SÃO PAULO,SP,SBMT,2010-10-01,14:50:00
16,39809,39809,INCIDENTE,SÃO PAULO,SP,NaN,2010-01-15,15:00:00
18,39828,39828,INCIDENTE,SANTOS,SP,NaN,2010-01-15,17:45:00
...,...,...,...,...,...,...,...,...
5735,79799,79799,INCIDENTE,MUITOS CAPÕES,RS,NaN,2020-12-21,13:30:00
5737,79748,79748,ACIDENTE,SANTANA DO LIVRAMENTO,RS,NaN,2020-12-23,13:30:00
5742,79800,79800,INCIDENTE,SÃO PAULO,SP,SBMT,2020-12-28,10:15:00
5747,79804,79804,INCIDENTE,CAMPINAS,SP,SBKP,2020-12-29,19:00:00


In [74]:
# ex. todas as ocorrências cuja cidade CONTÉM 'S' ou 'PA' (em qualquer posição)
filtro = df.ocorrencia_uf.str.contains('S | PA')
df.loc[filtro]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora


In [75]:
# ocorrencias de 2015
# ocorrencia_dia é do tipo DateTime
filtro = df.ocorrencia_dia.dt.year == 2015
df.loc[filtro] 

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora
2996,52992,52992,INCIDENTE GRAVE,SALVADOR,BA,SBSV,2015-01-01,11:40:00
2997,52979,52979,ACIDENTE,IVINHEMA,MS,NaN,2015-02-01,14:50:00
2998,53073,53073,INCIDENTE,TEFÉ,AM,SBTF,2015-02-01,16:28:00
2999,53074,53074,INCIDENTE,BELO HORIZONTE,MG,SBBH,2015-02-01,20:34:00
3000,52976,52976,ACIDENTE,TOLEDO,PR,SBTD,2015-04-01,22:04:00
...,...,...,...,...,...,...,...,...
3462,60632,60632,INCIDENTE GRAVE,ITABERÁ,SP,NaN,2015-12-24,14:00:00
3463,60600,60600,INCIDENTE,GUARULHOS,SP,SBGR,2015-12-25,19:00:00
3464,60642,60642,INCIDENTE,SÃO FRANCISCO DO SUL,SC,SSSS,2015-12-26,16:00:00
3465,60631,60631,ACIDENTE,MAÇAMBARÁ,RS,NaN,2015-12-28,19:00:00


In [76]:
''' forma mais compacta do código:

# ocorrencias de dezembro de 2016
filtro1 = (df.ocorrencia_dia.dt.year == 2016) & (df.ocorrencia_dia.dt.month == 12)
df.loc[filtro]
'''
# ocorrencias de dezembro de 2016
filtro1 = df.ocorrencia_dia.dt.year == 2016
filtro2 = df.ocorrencia_dia.dt.month == 12
df.loc[filtro1 & filtro2]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora
3481,60664,60664,ACIDENTE,MONTES CLAROS DE GOIÁS,GO,NaN,2016-12-01,14:00:00
3527,60739,60739,INCIDENTE,CASCAVEL,PR,SBCA,2016-12-02,18:06:00
3561,60872,60872,ACIDENTE,SALTO DE PIRAPORA,SP,SDBN,2016-12-03,16:30:00
3606,60992,60992,INCIDENTE,GUARULHOS,SP,SBGR,2016-12-04,11:30:00
3662,65330,65330,INCIDENTE GRAVE,SANTO ANTÔNIO DO LEVERGER,MT,SWLV,2016-12-06,15:00:00
3663,65367,65367,ACIDENTE,MARÍLIA,SP,NaN,2016-12-06,14:35:00
3685,65412,65412,INCIDENTE GRAVE,CAMPO GRANDE,MS,SBCG,2016-12-07,17:24:00
3686,65422,65422,INCIDENTE,BARRETOS,SP,SNBA,2016-12-07,17:00:00
3712,65512,65512,INCIDENTE,LONDRINA,PR,SBLO,2016-12-08,14:30:00
3713,65518,65518,INCIDENTE,FORTALEZA,CE,SBFZ,2016-12-08,15:03:00


In [82]:
# ocorrencias de julho de 2014 entre os dias 5 e 10
filtroAno = df.ocorrencia_dia.dt.year == 2014
filtroMes = df.ocorrencia_dia.dt.month == 7
filtroDia = (df.ocorrencia_dia.dt.day > 4) & (df.ocorrencia_dia.dt.day < 11)
df.loc[filtroAno & filtroMes & filtroDia]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora
2658,51460,51460,INCIDENTE,BRASÍLIA,DF,SBBR,2014-07-05,20:09:00
2701,51416,51416,ACIDENTE,ARUANÃ,GO,NaN,2014-07-06,4:27:00
2702,51517,51517,ACIDENTE,NOVA ANDRADINA,MS,NaN,2014-07-06,11:00:00
2703,51531,51531,ACIDENTE,MAIRIPORÃ,SP,NaN,2014-07-06,14:50:00
2704,51532,51532,ACIDENTE,ITAITUBA,PA,NaN,2014-07-06,13:00:00
2754,51756,51756,INCIDENTE GRAVE,ANÁPOLIS,GO,NaN,2014-07-07,12:46:00
2835,52279,52279,ACIDENTE,TANGARÁ DA SERRA,MT,SDVI,2014-07-09,16:00:00
2885,52718,52718,INCIDENTE,SALVADOR,BA,SBSV,2014-07-10,14:40:00
2886,52719,52719,INCIDENTE,PORTO ALEGRE,RS,SBPA,2014-07-10,21:40:00


In [85]:
# ocorrencias de julho de 2014 entre meia noite do dia 5 e 10h da manhã do dia 7

# unindo as colunas de data e horário 
# utilizando astype para não dar erro nos tipos de dado e utilizando to_datetime
# para que tenhas células com datetime, não com strings
df['ocorrencia_dia_hora'] = pd.to_datetime(df.ocorrencia_dia.astype(str) + ' ' + df.ocorrencia_hora)

filtro1 = df.ocorrencia_dia_hora >= '2014-07-05 00:00:00'
filtro2 = df.ocorrencia_dia_hora <= '2014-07-07 10:00:00'
df.loc[filtro1 & filtro2]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,ocorrencia_dia_hora
2658,51460,51460,INCIDENTE,BRASÍLIA,DF,SBBR,2014-07-05,20:09:00,2014-07-05 20:09:00
2701,51416,51416,ACIDENTE,ARUANÃ,GO,NaN,2014-07-06,4:27:00,2014-07-06 04:27:00
2702,51517,51517,ACIDENTE,NOVA ANDRADINA,MS,NaN,2014-07-06,11:00:00,2014-07-06 11:00:00
2703,51531,51531,ACIDENTE,MAIRIPORÃ,SP,NaN,2014-07-06,14:50:00,2014-07-06 14:50:00
2704,51532,51532,ACIDENTE,ITAITUBA,PA,NaN,2014-07-06,13:00:00,2014-07-06 13:00:00


In [90]:
# agrupamento, como fazer

# atribuindo filtros a um novo dataframe --> informações sobre março de 2010
filtro1 = df.ocorrencia_dia.dt.year == 2010
filtro2 = df.ocorrencia_dia.dt.month == 3
df201503 = df.loc[filtro1 & filtro2]
df201503

df201503.count()
# agrupando a partir de uma coluna e contando a partir de outra que não tenha dados nulos
df201503.groupby(['codigo_ocorrencia']).ocorrencia_hora.count()

# outra opção -- utilizar size() para agrupar e contar as linhas que foram agrupadas
df201503.groupby(['codigo_ocorrencia']).size()


df201503.groupby(['ocorrencia_classificacao']).size().sort_values()                    # ordem crescente
df201503.groupby(['ocorrencia_classificacao']).size().sort_values(ascending = False)   # ordem decrescente

######
# EX. agrupamento da região sudeste no ano de 2011
filtro1 = df.ocorrencia_dia.dt.year == 2011
filtro2 = df.ocorrencia_uf.isin(['SP', 'RJ', 'MG', 'ES'])
dfsudeste2011 = df.loc[filtro1 & filtro2] 
dfsudeste2011

# agrupando por tipo de ocorrência, descobrimos a quantidade de incidentes no 
# sudeste em 2011 e quantos de cada tipo
dfsudeste2011.groupby(['ocorrencia_classificacao']).size()

# agrupando por tipo de ocorrência e depois por uf para saber a quantidade de cada tipo
# de incidente
dfsudeste2011.groupby(['ocorrencia_classificacao', 'ocorrencia_uf']).size()

# agrupando por uf e depois por tipo de ocorrência para saber a quantidade de cada tipo
# de incidente
dfsudeste2011.groupby(['ocorrencia_uf', 'ocorrencia_classificacao']).size()

# filtrando as ocorrência identificadas na cidade do Rio de Janeiro
filtro = dfsudeste2011.ocorrencia_cidade == 'RIO DE JANEIRO'
dfsudeste2011.loc[filtro]

# agrupando o total de recomendações por mês e por cidade
# filtro = dfsudeste2011.total_recomendacoes > 0
# dfsudeste2011.loc[filtro].groupby(['ocorrencia_cidade', dfsudeste2011.ocorrencia_dia.dt.month]).total_recomendacoes.sum()

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,ocorrencia_dia_hora
554,43643,43643,INCIDENTE,RIO DE JANEIRO,RJ,SBJR,2011-02-01,17:18:00,2011-02-01 17:18:00
561,43715,43715,INCIDENTE,RIO DE JANEIRO,RJ,NaN,2011-06-01,19:58:00,2011-06-01 19:58:00
566,43652,43652,INCIDENTE,RIO DE JANEIRO,RJ,NaN,2011-09-01,21:30:00,2011-09-01 21:30:00
569,43829,43829,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,2011-09-01,18:15:00,2011-09-01 18:15:00
594,43831,43831,INCIDENTE,RIO DE JANEIRO,RJ,NaN,2011-01-24,17:30:00,2011-01-24 17:30:00
606,43820,43820,INCIDENTE,RIO DE JANEIRO,RJ,NaN,2011-01-30,17:41:00,2011-01-30 17:41:00
610,43828,43828,INCIDENTE,RIO DE JANEIRO,RJ,SBGL,2011-01-31,20:32:00,2011-01-31 20:32:00
618,43869,43869,ACIDENTE,RIO DE JANEIRO,RJ,NaN,2011-05-02,17:15:00,2011-05-02 17:15:00
619,43984,43984,INCIDENTE GRAVE,RIO DE JANEIRO,RJ,SBJR,2011-05-02,17:41:00,2011-05-02 17:41:00
622,43894,43894,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,2011-06-02,19:13:00,2011-06-02 19:13:00
